In [74]:
from ipythonblocks import BlockGrid
from IPython.display import clear_output
from collections import deque
from heapq import heappop, heappush
import random
import time

######################################################
############### Constants ############################

# display constants (for command line)
W = chr(9618)
B = chr(32)

# maze size settings
MAZE_ROWS = 10
MAZE_COLS = 14
######################################################
######################################################

In [75]:
maze_map = [
    [W, W, W, W, W, W, W, W, W, W, W, W, W, W],
    [W, B, B, B, B, W, W, W, B, B, B, W, B, W],
    [W, B, W, W, B, W, W, B, B, W, B, W, B, W],
    [W, B, W, B, B, B, B, B, W, W, B, B, B, W],
    [W, W, W, B, W, W, W, W, W, W, W, W, W, W],
    [W, W, B, B, B, B, W, B, W, B, W, B, B, W],
    [W, W, B, W, W, B, W, B, W, B, B, B, W, W],
    [W, W, B, W, W, W, W, B, B, B, W, W, W, W],
    [W, B, B, B, B, B, B, B, W, B, B, B, B, W],
    [W, W, W, W, W, W, W, W, W, W, W, W, W, W],
]

h_map = [
    [W,  W,  W,  W,  W,  W,  W,  W,  W,  W,  W,  W,  W,  W],
    [W, 25, 24, 23, 22,  W,  W,  W, 26, 27, 28,  W, 34,  W],
    [W, 28,  W,  W, 21,  W,  W, 24, 25,  W, 29,  W, 33,  W],
    [W, 27,  W, 19, 20, 21, 22, 23,  W,  W, 30, 31, 32,  W],
    [W,  W,  W, 18,  W,  W,  W,  W,  W,  W,  W,  W,  W,  W],
    [W,  W, 16, 17, 18, 19,  W,  9,  W,  5,  W,  1,  0,  W],
    [W,  W, 15,  W,  W, 20,  W,  8,  W,  4,  3,  2,  W,  W],
    [W,  W, 14,  W,  W,  W,  W,  7,  6,  5,  W,  W,  W,  W],
    [W, 14, 13, 12, 11, 10,  9,  8,  W,  6,  7,  8,  9,  W],
    [W,  W,  W,  W,  W,  W,  W,  W,  W,  W,  W,  W,  W,  W],
]

In [99]:
mr, mc, cr, cc = 1, 1, 5, 12 # mouse and cheese starting locations

In [77]:
def found_cheese(mr, mc, cr, cc):
    return((mr == cr) and (mc == cc))

In [78]:
def draw_maze(mc, mr, cr, cc, visited = set(), sol = set()):
    
    maze = BlockGrid(MAZE_COLS, MAZE_ROWS, fill=(200, 200, 200))
    clear_output()

    for block in maze:

        if maze_map[block.row][block.col] == W:
            block.red = 50
            block.green = 50
            block.blue = 50
        
        if (block.row, block.col) in visited:
            block.red = 0
            block.green = 0
            block.blue = 255
            
        if (block.row, block.col) in sol:
            block.red = 255
            block.green = 0
            block.blue = 255
            
    maze[cr, cc].set_colors(245, 178, 34) 
    maze[mr, mc].set_colors(115, 42, 42)
       
    maze.show()


In [79]:
def hitwall(mr, mc):
    return(maze_map[mr][mc] == W)

In [80]:
class Node:
    def __init__(self, location, parent = None):
        self.location = location
        self.parent = parent

In [81]:
class Queue:
    def __init__(self, *elements):
        self._elements = deque(elements)

    def enqueue(self, element):
        self._elements.append(element)

    def dequeue(self):
        return self._elements.popleft()

In [82]:
class PriorityQueue:
    def __init__(self, *elements):
        self._elements = [elements]

    def enqueue(self, priority, value):
        heappush(self._elements, (priority, value))

    def dequeue(self):
        return heappop(self._elements)

In [83]:
def move_mouse(mr, mc, move):
    
    if move == 'u':
        if hitwall(mr - 1, mc):
            ouch()
        else:
            mr = mr - 1
    elif move == 'd':
        if hitwall(mr + 1, mc):
            ouch()
        else:
            mr = mr + 1
    elif move == 'l':
        if hitwall(mr, mc - 1):
            ouch()
        else:
            mc = mc - 1
    elif move == 'r':
        if hitwall(mr, mc + 1):
            ouch()
        else:
            mc = mc + 1
    else:
        print("Invalid move")
    
    return(mr, mc)
    

In [84]:
def sol_path(visited):
    not_root = True
    solution = []
    child = (cr, cc)
    while not_root:
        solution.append(visited[child])
        if solution[-1] == None:
            not_root = False
        else:
            child = visited[child]
    return solution

In [85]:
def ouch():
    print("bump")

In [86]:
def random_agent(mr, mc, cr, cc):
    
    moves = ['u', 'd', 'l', 'r']
    
    draw_maze(mc, mr, cr, cc)
    
    while not found_cheese(mr, mc, cr, cc):
        
        move = random.choice(moves)
        mr, mc = move_mouse(mr, mc, move)
        draw_maze(mc, mr, cr, cc)
        time.sleep(.5)
        
    print("Cheese found!")
    print("Total path cost: {}".format(action_cost))


In [87]:
def user_move(mr, mc, cr, cc):
    
    action_cost = 0
    draw_maze(mc, mr, cr, cc)
    print("You may move the mouse with the following moves:\nUp: u\nDown: d\nLeft: l\nRight: r\nTo exit, press x.")
        
    while not found_cheese(mr, mc, cr, cc):
            
            move = input("Input move: ")
                    
            if move == 'x':
                break
            else:
                mr, mc = move_mouse(mr, mc, move)
                    
            action_cost += 1
            draw_maze(mc, mr, cr, cc)
            print("You may move the mouse with the following moves:\nUp: u\nDown: d\nLeft: l\nRight: r\nTo exit, press x.")
 
    draw_maze(mc, mr, cr, cc)
    if move == 'x':
        print("User exited the maze.")
    else:
        print("You found the cheese!")
    print("Total path cost: {}".format(action_cost))

                 

In [88]:
def depth_first_search(mr, mc, cr, cc):
    
    moves = ['u', 'd', 'l', 'r']
    action_cost = 0
    curr_node = Node((mr, mc))
    frontier = [curr_node]
    visited = dict()
    draw_maze(mc, mr, cr, cc, visited)
    while frontier:
        time.sleep(.5)
        curr_node = frontier.pop()
        mr, mc = curr_node.location[0], curr_node.location[1]
        if not found_cheese(mr, mc, cr, cc):
            if curr_node.location not in visited:
                visited.update({curr_node.location : curr_node.parent})
                for move in moves:
                    move_r, move_c = move_mouse(mr, mc, move)
                    if not ((mr == move_r) and (mc == move_c)) and (move_r, move_c) not in visited:
                        frontier.append(Node((move_r, move_c), (mr, mc)))
                action_cost += 1
        else:
            visited.update({curr_node.location : curr_node.parent})
            draw_maze(mc, mr, cr, cc, visited, sol_path(visited))
            print("Cheese found!")
            print("Total path cost: {}".format(action_cost))
            break
            
        draw_maze(mc, mr, cr, cc, visited)
        print("Total path cost: {}".format(action_cost))
        

In [89]:
def breadth_first_search(mr, mc, cr, cc):
    
    moves = ['u', 'd', 'l', 'r']
    action_cost = 0
    curr_node = Node((mr, mc))
    frontier = None # Fill this in
    visited = dict()
    draw_maze(mc, mr, cr, cc, visited)
    while frontier:
        time.sleep(.5)
        curr_node = None # Fill this in
        mr, mc = None # Fill this in
        if not found_cheese(mr, mc, cr, cc):
            None # Fill this in
        else:
            visited.update({curr_node.location : curr_node.parent})
            draw_maze(mc, mr, cr, cc, visited, sol_path(visited))
            print("Cheese found!")
            print("Total path cost: {}".format(action_cost))
            break
            
        draw_maze(mc, mr, cr, cc, visited)
        print("Total path cost: {}".format(action_cost))

In [90]:
def best_first_search(mr, mc, cr, cc):
    
    moves = ['u', 'd', 'l', 'r']
    action_cost = 0
    curr_node = Node((mr, mc))
    frontier = None # Fill this in
    visited = dict()
    draw_maze(mc, mr, cr, cc, visited)
    while frontier:
        time.sleep(.5)
        curr_node = None # Fill this in
        mr, mc = None #Fill this in
        if not found_cheese(mr, mc, cr, cc):
            None # Fill this in
        else:
            visited.update({curr_node.location : curr_node.parent})
            draw_maze(mc, mr, cr, cc, visited, sol_path(visited))
            print("Cheese found!")
            print("Total path cost: {}".format(action_cost))
            break
            
        draw_maze(mc, mr, cr, cc, visited)
        print("Total path cost: {}".format(action_cost))

In [98]:
def run_maze():
    
    run_choice = input(
        "Press 1 to run the depth first mouse.\n"
        "Press 2 to run the breadth first mouse.\n"
        "Press 3 to run the best first mouse.\n"
        "Press 4 to run the mouse manually.\n"
        "Press 5 to run the random mouse.\n"
        "Press 6 to exit the menu."
    )
    
    if run_choice == '1':
        print("made it")
        depth_first_search(mr, mc, cr, cc)
    elif run_choice == '2':
        breadth_first_search(mr, mc, cr, cc)
    elif run_choice == '3':
        best_first_search(mr, mc, cr, cc)
    elif run_choice == '4':
        user_move(mr, mc, cr, cc)
    elif run_choice == '5':
        random_agent(mr, mc, cr, cc)
    else:
        return


In [96]:
run_maze()

,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
,,,,,,,,,,,,,
